In [5]:
import sqlalchemy as sql
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, scoped_session
from sqlalchemy import Column, Integer, String, BigInteger, VARCHAR, Float, ForeignKey
import datetime 
import time
import os

pd.set_option('display.max_rows', 100)

In [6]:
    ss_data_cols = """Name
    Team
    Opponent
    Position
    Projection
    Actual
    Price
    Value
    proj_own
    dk_point
    dk_25_percentile
    dk_50_percentile
    dk_75_percentile
    dk_85_percentile
    dk_95_percentile
    dk_99_percentile
    fd_points
    fd_25_percentile
    fd_50_percentile
    fd_75_percentile
    fd_85_percentile
    fd_95_percentile
    fd_99_percentile
    fdraft_points
    fdraft_25_percentile
    fdraft_50_percentile
    fdraft_75_percentile
    fdraft_85_percentile
    fdraft_95_percentile
    fdraft_99_percentile
    yahoo_points
    yahoo_25_percentile
    yahoo_50_percentile
    yahoo_75_percentile
    yahoo_85_percentile
    yahoo_95_percentile
    yahoo_99_percentile
    dk_std
    fd_std
    fdraft_std
    yahoo_std
    pass_attempts
    completions
    passing_yards
    passing_td
    receptions
    receiving_yards
    receiving_td
    rushes
    rushing_yards
    rushing_td""".split('\n')
    
    

    _4f4_proj_cols = """Season 
    Week
    PID
    Player
    Pos
    Team
    Opp
    aFPA
    aFPA_Rk
    FFPts
    Comp
    Pass_Att
    Pass_Yds
    Pass_TD
    INT
    Rush_Att
    Rush_Yds
    Rush_TD
    Rec
    Rec_Yds
    Rec_TD
    Pa1D
    Ru1D
    Rec1D
    Fum
    XP
    FG
    Grade""".split('\n')
    
    _4f4_ceil_cols = """Season
    Week
    PID
    Player
    Pos
    Team
    Opp
    aFPA
    aFPA_Rk
    FFPts
    Comp
    Pass_Att
    Pass_Yds
    Pass_TD
    INT
    Rush_Att
    Rush_Yds
    Rush_TD
    Rec
    Rec_Yds
    Rec_TD
    Pa1D
    Ru1D
    Rec1D
    Fum
    XP
    FG
    Grade
    DK_Proj
    DK_Price
    DK_Val
    DK_Pts_per_1k
    DK_Flr
    DK_Flr_Val
    DK_Flr_per_1k
    DK_Ceil
    DK_Ceil_Val
    DK_Ceil_per_1k
    DK_Wk_Change
    DK_Own_Percent
    FD_Proj
    FD_Price
    FD_Val
    FD_Pts_pr_1k
    FD_Flr
    FD_Flr_Val
    FD_Flr_pr_1k
    FD_Ceil
    FD_Ceil_Val
    FD_Ceil_pr_1k
    FD_Wk_Change
    FD_Own_Percent
    Y_Proj
    Y_Price
    Y_Val
    Y_Pt_pr_Dollar
    Y_Flr
    Y_Flr_Val
    Y_Floor_pr_Dollar
    Y_Ceil
    Y_Ceil_Val
    Y_Ceil_pr_Dollar
    Y_Wk_Change
    FyD_Proj
    FyD_Price
    FyD_Val
    FyD_Pt_pr_1k
    FyD_Flr
    FyD_Flr_Val
    FyD_Flr_pr_1k
    FyD_Ceil
    FyD_Ceil_Val
    FyD_Ceil_pr_1k
    FyD_Wk_Change
    O_U
    Line
    Team_O_U""".split('\n')


In [18]:
SQLALCHEMY_DATABASE_URL = f"sqlite+pysqlite:///DFS/Resources/DFS.db"

engine = create_engine(SQLALCHEMY_DATABASE_URL, pool_pre_ping=True, connect_args={"check_same_thread": True})

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
db_session = scoped_session(SessionLocal)
Base = declarative_base(bind=engine)

Base.metadata.create_all(engine)

In [12]:
class DictMixIn:
    def to_dict(self):
        return {
            column.name: getattr(self, column.name)
            if not isinstance(getattr(self, column.name), datetime.datetime)
            else getattr(self, column.name).isoformat()
            for column in self.__table__.columns
        }
    
class Players(Base, DictMixIn):
    __tablename__ = 'players'
    PID = Column(String(25), primary_key=True)
    Name = Column(String(25))
    Position = Column(String)
    Team = Column(String)
    
class SS_Data(Base, DictMixIn):
    __tablename__ = 'Saber_Sim_Data'
#     PID = Column(String, ForeignKey('players.PID'))
    Name = Column(String(30), primary_key=True)
    Team = Column(String(10), primary_key=True)
    Opponent = Column(String(10), primary_key=True)
    Position = Column(String(5), primary_key=True)
    Projection = Column(Float, primary_key=True)
    Actual = Column(String)
    Price = Column(BigInteger)
    Value = Column(Float)
#     ## added by me to keep track of week/Season
#     Date_Added = Column(Date) 
#     Week = Column(Integer)
#     Season = Column(Integer)
#     ## end of added data points
    proj_own = Column(Float)
    dk_point = Column(Float)
    dk_25_percentile = Column(Float)
    dk_50_percentile  = Column(Float)
    dk_75_percentile = Column(Float) 
    dk_85_percentile = Column(Float)
    dk_95_percentile = Column(Float) 
    dk_99_percentile = Column(Float)  
    fd_points = Column(Float) 
    fd_25_percentile = Column(Float)
    fd_50_percentile = Column(Float) 
    fd_75_percentile = Column(Float) 
    fd_85_percentile = Column(Float)
    fd_95_percentile  = Column(Float)
    fd_99_percentile = Column(Float) 
    fdraft_points = Column(Float)
    fdraft_25_percentile = Column(Float) 
    fdraft_50_percentile = Column(Float) 
    fdraft_75_percentile = Column(Float)
    fdraft_85_percentile = Column(Float) 
    fdraft_95_percentile = Column(Float) 
    fdraft_99_percentile = Column(Float)
    yahoo_points = Column(Float)
    yahoo_25_percentile = Column(Float) 
    yahoo_50_percentile = Column(Float)
    yahoo_75_percentile = Column(Float) 
    yahoo_85_percentile = Column(Float) 
    yahoo_95_percentile = Column(Float)
    yahoo_99_percentile = Column(Float)
    dk_std = Column(Float)
    fd_std = Column(Float)
    fdraft_std = Column(Float)
    yahoo_std = Column(Float)
    pass_attempts = Column(Float)
    completions = Column(Float)
    passing_yards = Column(Float)
    passing_td = Column(Float)
    receptions = Column(Float)
    receiving_yards = Column(Float)
    receiving_td = Column(Float)
    rushes = Column(Float)
    rushing_yards = Column(Float)
    rushing_td = Column(Float)
    
class _4f4_Proj(Base, DictMixIn):
    __tablename__ = '4for4_Projections'
    Season  = Column(Integer, primary_key=True)
    Week = Column(Integer, primary_key=True)
    PID = Column(String(20), primary_key=True)
    Player = Column(String(20), primary_key=True)
    Pos = Column(String(20), primary_key=True)
    Team = Column(String(20), primary_key=True)
    Opp = Column(String(20))
    aFPA = Column(Float)
    aFPA_Rk = Column(Integer)
    FFPts = Column(Float)
    Comp = Column(Float)
    Pass_Att = Column(Float)
    Pass_Yds = Column(Float)
    Pass_TD = Column(Float)
    INT = Column(Float)
    Rush_Att = Column(Float)
    Rush_Yds = Column(Float)
    Rush_TD = Column(Float)
    Rec = Column(Float)
    Rec_Yds = Column(Float)
    Rec_TD = Column(Float)
    Pa1D = Column(Float)
    Ru1D = Column(Float)
    Rec1D = Column(Float)
    Fum = Column(Float)
    XP = Column(Float)
    FG = Column(Float)
    Grade = Column(String)
    
class _4f4_Ceiling(Base, DictMixIn):
    __tablename__ = '4for4_Ceiling'
    Season = Column(Integer, primary_key=True)
    Week = Column(Integer, primary_key=True)
    PID = Column(Integer, primary_key=True)
    Player = Column(String(30), primary_key=True)
    Pos = Column(String(30), primary_key=True)
    Team = Column(String(30), primary_key=True)
    Opp = Column(String(30))
    aFPA = Column(Float)
    aFPA_Rk = Column(Integer)
    FFPts = Column(Float)
    Comp = Column(Float)
    Pass_Att = Column(Float)
    Pass_Yds = Column(Float)
    Pass_TD = Column(Float)
    INT = Column(Float)
    Rush_Att = Column(Float)
    Rush_Yds = Column(Float)
    Rush_TD = Column(Float)
    Rec = Column(Float)
    Rec_Yds = Column(Float)
    Rec_TD = Column(Float)
    Pa1D = Column(Float)
    Ru1D = Column(Float)
    Rec1D = Column(Float)
    Fum = Column(Float)
    XP = Column(Float)
    FG = Column(Float)
    Grade = Column(Integer)
    DK_Proj = Column(Float)
    DK_Price = Column(Float)
    DK_Val = Column(Float)
    DK_Pts_per_1k = Column(Float)
    DK_Flr = Column(Float)
    DK_Flr_Val = Column(Float)
    DK_Flr_per_1k = Column(Float)
    DK_Ceil = Column(Float)
    DK_Ceil_Val = Column(Float)
    DK_Ceil_per_1k = Column(Float)
    DK_Wk_Change = Column(Integer)
    DK_Own_Percent = Column(Integer)
    FD_Proj = Column(Float)
    FD_Price = Column(Float)
    FD_Val = Column(Float)
    FD_Pts_pr_1k = Column(Float)
    FD_Flr = Column(Float)
    FD_Flr_Val = Column(Float)
    FD_Flr_pr_1k = Column(Float)
    FD_Ceil = Column(Float)
    FD_Ceil_Val = Column(Float)
    FD_Ceil_pr_1k = Column(Float)
    FD_Wk_Change = Column(String)
    FD_Own_Percent = Column(String)
    Y_Proj = Column(Float)
    Y_Price = Column(Float)
    Y_Val = Column(Float)
    Y_Pt_pr_Dollar = Column(Float)
    Y_Flr = Column(Float)
    Y_Flr_Val = Column(Float)
    Y_Floor_pr_Dollar = Column(Float)
    Y_Ceil = Column(Float)
    Y_Ceil_Val = Column(Float)
    Y_Ceil_pr_Dollar = Column(Float)
    Y_Wk_Change = Column(Integer)
    FyD_Proj = Column(Float)
    FyD_Price = Column(Float)
    FyD_Val = Column(Float)
    FyD_Pt_pr_1k = Column(Float)
    FyD_Flr = Column(Float)
    FyD_Flr_Val = Column(Float)
    FyD_Flr_pr_1k = Column(Float)
    FyD_Ceil = Column(Float)
    FyD_Ceil_Val = Column(Float)
    FyD_Ceil_pr_1k = Column(Float)
    FyD_Wk_Change = Column(String)
    O_U = Column(Float)
    Line = Column(Float)
    Team_O_U = Column(Float)


In [13]:
workbook_path = os.path.join('DFS', 'Resources', 'DraftKingsNoNotes.xlsx')

workbook = pd.ExcelFile(workbook_path)

ss_data = workbook.parse('SS Data')
_4f4_proj = workbook.parse('4f4 Proj Data')
_4f4_ceil = workbook.parse('4f4 Ceiling Data')
_4f4_wrcb= workbook.parse('4f4 WRCB')
_4f4_redZ = workbook.parse('4f4 Red Z Data')
_4f4_wr_fp = workbook.parse('4f4 WR FP Last 4 ')
_4f4_rb_fp = workbook.parse('4f4 RB FP Last 3')
_4f4_rb_tar = workbook.parse('4f4 RB TAR Last 3')
airy_wr = workbook.parse('AirY WR Last 4').drop(['Unnamed: 0'], axis=1)
airy_te = workbook.parse('AirY TE Last 3').drop(['Unnamed: 0'], axis=1)
etr_ceiling = workbook.parse('ETR Ceiling Data')

In [57]:
airy_wr

,full_name,position,team,targets,rec,rec_yards,air_yards,yac,td,adot,racr,ms_air,tgt_share,wopr,ppr,Tar/Gam,AY/Gam
0,Michael Thomas,WR,NO,46.0,39.0,411.0,369.0,110.0,3.0,8.0,1.11,0.37,0.33,0.75,98.1,11.50,92.25
1,A.J. Brown,WR,TEN,29.0,20.0,447.0,378.0,232.0,4.0,13.0,1.18,0.39,0.28,0.69,90.0,7.25,94.50
2,Allen Robinson,WR,CHI,44.0,26.0,390.0,576.0,104.0,4.0,13.1,0.68,0.40,0.27,0.69,89.0,11.00,144.00
3,D.J. Moore,WR,CAR,39.0,24.0,395.0,511.0,118.0,3.0,13.1,0.77,0.40,0.24,0.63,83.0,9.75,127.75
4,DeAndre Hopkins,WR,HOU,37.0,24.0,397.0,460.0,127.0,3.0,12.4,0.86,0.33,0.28,0.65,82.9,9.25,115.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00
195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00
196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00
197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00


In [15]:
ss_data.columns = ss_data_cols
_4f4_proj.columns = _4f4_proj_cols
_4f4_ceil.columns = _4f4_ceil_cols

In [16]:
def find_unique_players(df, current_players):
    if 'Player' in df.columns:
        name_key = 'Player'
    elif 'Name' in df.columns:
        name_key = 'Name'
    elif 'full_name' in df.columns:
        name_key = 'full_name'
            
    names = df[name_key]
    
    new_players=[]
    
    for name in names:
        if name in current_players:
            return None
        
        elif name not in current_players:
            if 'PID' in df.columns:
                df_new_player = df[df[name_key].str.contains(name)]

                new_player = Player({
                    'PID' : df_new_player['PID'],
                    'Name' : df_new_player[name_key],
                    'Position' : df_new_player['Position'],
                    'Team' : df_new_player['Team']
                                   })
                new_players.append(new_player)

            elif 'PID' not in df.columns:
                df_new_player = df[df[name_key].str.contains(name)]
                
                player_id = df_new_player[name_key].str.split(' ')[1]

                new_player = Player({
                    'PID' : player_id,
                    'Name' : df_new_player[name_key],
                    'Position' : df_new_player['Position'],
                    'Team' : df_new_player['Team']
                                   })
                    
                new_players.append(new_player)
                
    return new_players
                    
        
    

In [70]:
ss_data.to_sql(name='Saber_Sim_Data',
                con=engine,
                if_exists='replace',
                index=True,
                chunksize=200)

_4f4_proj.to_sql(name='4for4_Projections',
                con=engine,
                if_exists='replace',
                index=True,
                chunksize=200)
_4f4_ceiling.to_sql(name='4for4_Ceilings',
                con=engine,
                if_exists='replace',
                index=True,
                chunksize=200)
# _4f4_wrcb.to_sql(name='4for4_wrcb',
#                 con=engine,
#                 if_exists='replace',
#                 index=True,
#                 chunksize=200)
# _4f4_redZ.to_sql(name='4for4_Red_zone',
#                 con=engine,
#                 if_exists='replace',
#                 index=True,
#                 chunksize=200)
# _4f4_wr_fp.to_sql(name='4for4_WR_FP',
#                 con=engine,
#                 if_exists='replace',
#                 index=True,
#                 chunksize=200)
# _4f4_rb_fp.to_sql(name='4for4_RB_FP',
#                 con=engine,
#                 if_exists='replace',
#                 index=True,
#                 chunksize=200)
# _4f4_rb_tar.to_sql('4for4_RB_TAR',
#                 con=engine,
#                 if_exists='replace',
#                 index=True,
#                 chunksize=200)
# airy_wr.to_sql(name='AirY_WR',
#                 con=engine,
#                 if_exists='replace',
#                 index=True,
#                 chunksize=200)
# airy_te.to_sql(name='AirY_TE',
#                 con=engine,
#                 if_exists='replace',
#                 index=True,
#                 chunksize=200)
# etr_ceiling.to_sql(name='ETR_Ceiling',
#                 con=engine,
#                 if_exists='replace',
#                 index=True,
#                 chunksize=200)

In [63]:
_4f4_wr_fp_cols = ['Player', 'Pos', 'Team', 'G', 'W5', 'W6', 'W7', 'W8', 'DK_Pts','DK_Pts_pr_G']
_4f4_rb_fp_cols = ['Player', 'Pos', 'Team', 'G', 'W5', 'W6', 'W7', 'W8', 'DK_Pts','DK_Pts_pr_G']
_4f4_rb_tar_cols = ['Player', 'Pos', 'Team', 'G', 'W13', 'W14', 'W15', 'TGTs', 'TGTs_G', 'Array']
airy_wr_cols = ['full_name', 'position', 'team', 'targets', 'rec', 'rec_yards',
                   'air_yards', 'yac', 'td', 'adot', 'racr', 'ms_air', 'tgt_share', 'wopr',
                   'ppr', 'Tar_pr_Gam', 'AY_pr_Gam']

airy_te_cols = ['full_name', 'position', 'team', 'targets', 'rec', 'rec_yards',
                   'air_yards', 'yac', 'td', 'adot', 'racr', 'ms_air', 'tgt_share', 'wopr',
                   'ppr', 'Tar_pr_3']

In [45]:
etr_ceiling.dtypes

Season              int64
Week                int64
PID                object
Player             object
Pos                object
Team               object
Opp                object
aFPA              float64
aFPA Rk             int64
FFPts             float64
Comp              float64
Pass Att          float64
Pass Yds          float64
Pass TD           float64
INT               float64
Rush Att          float64
Rush Yds          float64
Rush TD           float64
Rec               float64
Rec Yds           float64
Rec TD            float64
Pa1D              float64
Ru1D              float64
Rec1D             float64
Fum               float64
XP                float64
FG                float64
Grade              object
DK (Proj)         float64
DK ($)            float64
DK (Val)          float64
DK (Pt/$1k)       float64
DK (Flr)          float64
DK (Flr Val)      float64
DK (Flr/$1k)      float64
DK (Ceil)         float64
DK (Ceil Val)     float64
DK (Ceil/$1k)     float64
DK (Wk $ Chg